In [3]:
#prepare features
dic = {"fruit":[],"state":[],"filename":[]}

indexBin = [*range(0, 256, 1)]
indexBinr = ["r"+str(x) for x in indexBin]
indexBing = ["g"+str(x) for x in indexBin]
indexBinb = ["b"+str(x) for x in indexBin]
newIndexbin = indexBinr+indexBing+indexBinb
for key in newIndexbin :
    dic[key]= []




In [4]:
import cv2
import numpy as np
from scipy.ndimage import generic_filter
import os

def fruitToData(directory,fruit,state):
    global dic
    goodappledirectory = os.listdir(directory)

    #print(listOfFile)
    for goodapplefilename in goodappledirectory:

        #read image
        myimg = cv2.imread(directory+'/'+goodapplefilename)
        myimg = myimg.astype(np.float32)
        myimg [myimg >240]=-1
        (b,g,r) = cv2.split(myimg)

        dimension = myimg.shape[0]*myimg.shape[1]
        

        histR = cv2.calcHist([r], [0], None, [256], [0, 256]).ravel()
        #histR /= histR.sum()
        histG = cv2.calcHist([g], [0], None, [256], [0, 256]).ravel()
        #histG /= histG.sum()
        histB = cv2.calcHist([b], [0], None, [256], [0, 256]).ravel()
        #histB /= histB.sum()
        hist = np.concatenate([histR,histG,histB])
        hist = hist.astype(np.float32)
        hist = hist/dimension

        #hist = list(hist)
        dic["fruit"].append(fruit)
        dic["state"].append(state)
        dic["filename"].append(directory+'/'+goodapplefilename)
        keys = list(dic.keys())
        keys= keys[3:]
        #print(keys)
        print(len(dic["filename"]),(len(dic["fruit"])))
        for k in range(0,len(keys)):
            currentKey = keys[k]
            currentValue = hist[k]
            dic[currentKey].append(currentValue)
    



fruitToData("freshapples","apple","good")
fruitToData("freshbanana","banana","good")
fruitToData("freshoranges","oranges","good")
fruitToData("rottenapples","apple","bad")
fruitToData("rottenbanana","banana","bad")
fruitToData("rottenoranges","oranges","bad")

print(len(dic["filename"]),(len(dic["fruit"])))


1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
11 11
12 12
13 13
14 14
15 15
16 16
17 17
18 18
19 19
20 20
21 21
22 22
23 23
24 24
25 25
26 26
27 27
28 28
29 29
30 30
31 31
32 32
33 33
34 34
35 35
36 36
37 37
38 38
39 39
40 40
41 41
42 42
43 43
44 44
45 45
46 46
47 47
48 48
49 49
50 50
51 51
52 52
53 53
54 54
55 55
56 56
57 57
58 58
59 59
60 60
61 61
62 62
63 63
64 64
65 65
66 66
67 67
68 68
69 69
70 70
71 71
72 72
73 73
74 74
75 75
76 76
77 77
78 78
79 79
80 80
81 81
82 82
83 83
84 84
85 85
86 86
87 87
88 88
89 89
90 90
91 91
92 92
93 93
94 94
95 95
96 96
97 97
98 98
99 99
100 100
101 101
102 102
103 103
104 104
105 105
106 106
107 107
108 108
109 109
110 110
111 111
112 112
113 113
114 114
115 115
116 116
117 117
118 118
119 119
120 120
121 121
122 122
123 123
124 124
125 125
126 126
127 127
128 128
129 129
130 130
131 131
132 132
133 133
134 134
135 135
136 136
137 137
138 138
139 139
140 140
141 141
142 142
143 143
144 144
145 145
146 146
147 147
148 148
149 149
150 150
151 151
152 152


In [5]:
#!pip install pandas
#print(len(dic["filename"]))
import pandas as pd
df = pd.DataFrame.from_dict(dic)
df.to_csv(r'fruitdata.csv', index = False)



#print(df.iloc[:10,100:120])


# Categorizar por si esta podrido o no

In [14]:
#load dataframe
import pandas as pd

df = pd.read_csv("fruitdata.csv")
df.fruit= df.fruit.astype('category')
df.state = df.state.astype('category')
onlyappledf = df[df.fruit =="apple" ]
onlyappledf["state"] = onlyappledf["state"].map({"good": 1, "bad": 0})
#print (df.dtypes)
#print()


<ipython-input-14-e0009a80da42>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onlyappledf["state"] = onlyappledf["state"].map({"good": 1, "bad": 0})


In [15]:
import math
import numpy as np

class Knn:
    def __init__(self, data):
        self.data = data
     
    def classification(self, x,n):
        def distance(WorldPoint,currentPoint):
            vecxtov = [WorldPoint[i] - currentPoint[i] for i in range(len(WorldPoint))] 
            distacc = 0
            for pos in vecxtov:
                distacc += pos**2
            return math.sqrt(distacc)
        
        knndf  = self.data.copy()
        knndf["distance"] = knndf.iloc[:, 3:].apply(distance,currentPoint=x, axis=1)
        knndf = knndf.sort_values(by='distance', ascending=True).state
        
        knnselection = knndf.head(n)
        classification = knnselection.value_counts().index[0]
        
        keys = knnselection.value_counts().index.tolist()
        classes = knnselection.value_counts().tolist()
        classes = list(map(lambda x: x/n, classes))
        report = dict(zip(keys, classes))
        #print(report)
        '''print(knnselection.value_counts())
        print(knnselection.value_counts().tolist() )
        print(knnselection.value_counts().index.tolist() )'''
        
        return  classification,report

onlyappledf = onlyappledf.sample(frac = 1,random_state=23)
train=onlyappledf.sample(frac=0.90,random_state=108) #random state is a seed value
test=onlyappledf.drop(train.index)

fruitKnn = Knn(train)

AcurracyList = []
for i in range(len(test)):
    currentTest = test.iloc[[i]]
    currentStats = currentTest.values.tolist()[0][3:]
    neighboorhods = 5

    classification,report = fruitKnn.classification(currentStats,neighboorhods)
    print(report)
    realclass = currentTest.state.item()
    #print("predict: ",classification,"real",realclass)
    AcurracyList.append(classification==realclass)




{0: 1.0, 1: 0.0}
{0: 1.0, 1: 0.0}
{0: 1.0, 1: 0.0}
{1: 1.0, 0: 0.0}
{0: 0.6, 1: 0.4}
{0: 1.0, 1: 0.0}
{1: 1.0, 0: 0.0}
{1: 0.6, 0: 0.4}
{1: 1.0, 0: 0.0}
{0: 0.6, 1: 0.4}
{1: 0.6, 0: 0.4}
{0: 1.0, 1: 0.0}
{0: 1.0, 1: 0.0}
{0: 1.0, 1: 0.0}
{0: 1.0, 1: 0.0}
{1: 0.8, 0: 0.2}
{0: 1.0, 1: 0.0}
{0: 0.8, 1: 0.2}
{0: 1.0, 1: 0.0}
{0: 1.0, 1: 0.0}
{1: 0.8, 0: 0.2}
{0: 1.0, 1: 0.0}
{0: 1.0, 1: 0.0}
{1: 0.8, 0: 0.2}
{1: 1.0, 0: 0.0}
{1: 0.8, 0: 0.2}
{0: 0.8, 1: 0.2}
{0: 0.8, 1: 0.2}


In [3]:
NpAcurracyList = np.array(AcurracyList)
hits = len(NpAcurracyList[NpAcurracyList==True]) #Numero de aciertos
numberSamples = len(AcurracyList) #Numero de muestras
accuracy = hits/numberSamples
print(accuracy)

0.9787234042553191


# Categorizar por tipo de fruta

In [20]:
#load dataframe
import pandas as pd

df = pd.read_csv("fruitdata.csv")
df.fruit= df.fruit.astype('category')
df.state = df.state.astype('category')
df["fruit"] = df["fruit"].map({"apple": 0, "banana": 1,"oranges":2})

print( df.sample(frac = 0.2,random_state=23))

    fruit state                       filename        r0        r1        r2  \
586     1   bad   rottenbanana/banana (78).png  0.000158  0.000632  0.001571   
74      0  good   freshapples/manzana (77).png  0.000000  0.000000  0.000000   
73      0  good   freshapples/manzana (76).png  0.000000  0.000000  0.000000   
253     2  good  freshoranges/naranja (47).png  0.000000  0.000000  0.000000   
576     1   bad   rottenbanana/banana (69).png  0.000000  0.000000  0.000000   
..    ...   ...                            ...       ...       ...       ...   
46      0  good   freshapples/manzana (51).png  0.000000  0.000000  0.000000   
130     1  good    freshbanana/banana (39).png  0.000000  0.000000  0.000000   
581     1   bad   rottenbanana/banana (73).png  0.000012  0.000150  0.000349   
281     2  good  freshoranges/naranja (82).png  0.000000  0.000000  0.000000   
555     1   bad    rottenbanana/banana (5).png  0.002997  0.000000  0.000000   

           r3        r4        r5      

In [24]:
import math
import numpy as np

class Knn:
    def __init__(self, data):
        self.data = data
     
    def classification(self, x,n):
        def distance(WorldPoint,currentPoint):
            vecxtov = [WorldPoint[i] - currentPoint[i] for i in range(len(WorldPoint))] 
            distacc = 0
            for pos in vecxtov:
                distacc += pos**2
            return math.sqrt(distacc)
        
        knndf  = self.data.copy()
        knndf["distance"] = knndf.iloc[:, 3:].apply(distance,currentPoint=x, axis=1)
        knndf = knndf.sort_values(by='distance', ascending=True).fruit
        
        knnselection = knndf.head(n)
        classification = knnselection.value_counts().index[0]
        return  classification

df = df.sample(frac = 1,random_state=23)
train=df.sample(frac=0.98,random_state=109) #random state is a seed value
test=df.drop(train.index)

fruitKnn = Knn(train)

AcurracyList = []
for i in range(len(test)):
    currentTest = test.iloc[[i]]
    currentStats = currentTest.values.tolist()[0][3:]
    neighboorhods = 5

    classification = fruitKnn.classification(currentStats,neighboorhods)
    realclass = currentTest.fruit.item()
    print("predict: ",classification,"real",realclass)
    AcurracyList.append(classification==realclass)

768
<class 'list'>
predict:  0 real 0
768
<class 'list'>
predict:  1 real 0
768
<class 'list'>
predict:  0 real 0
768
<class 'list'>
predict:  0 real 0
768
<class 'list'>
predict:  2 real 2
768
<class 'list'>
predict:  1 real 1
768
<class 'list'>
predict:  1 real 1
768
<class 'list'>
predict:  1 real 1
768
<class 'list'>
predict:  0 real 0
768
<class 'list'>
predict:  0 real 0
768
<class 'list'>
predict:  1 real 1
768
<class 'list'>
predict:  0 real 1
768
<class 'list'>
predict:  0 real 0
768
<class 'list'>
predict:  1 real 1


In [23]:
NpAcurracyList = np.array(AcurracyList)
hits = len(NpAcurracyList[NpAcurracyList==True]) #Numero de aciertos
numberSamples = len(AcurracyList) #Numero de muestras
accuracy = hits/numberSamples
print(accuracy)

0.8285714285714286


1
